In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cater = pd.read_csv('catering_orders.csv')
vendors = pd.read_csv('vendors.csv', header=1)
venue = pd.read_csv('venue.csv', header=1)

coords = pd.read_csv('coords.csv')
coords = coords.rename(columns={'City': 'Municipality'})
vendors = vendors.merge(coords, on='Municipality', how='left')
vendors = vendors.dropna(subset=['Latitude', 'Longitude'])

In [3]:
cater.head()

,Company,Municipality,Estimated Price,Package,Dishes,Date,Rating
0,Lucban Culinary Events,Lucban,45120.0,Premium,"Adobo, Caldereta, Sinigang, Puto, Lechon",2022-03-15,4.7
1,Lopez Catering Corner,Lopez,39850.0,Buffet,"Halo-Halo, Menudo, Lumpiang Shanghai, Laing, P...",2023-07-22,4.5
2,Gumaca Feast Caterers,Gumaca,52030.0,Standard,"Seafood Paella, Bicol Express, Inihaw na Baboy...",2024-11-10,4.2
3,Sariaya Food Masters,Sariaya,47890.0,Budget,"Fruit Salad, Buttered Shrimp, Dinuguan, Embuti...",2025-01-19,4.9
4,Pagbilao Catering Services,Pagbilao,43500.0,Plated,"Palabok, Crispy Pata, Longganisa, Tokwa't Babo...",2023-09-03,4.6


In [4]:
vendors.head()

,Category,Vendor Name,Municipality,Estimated Price,Package,Date,Rating,Latitude,Longitude
0,Photographer,Golden Lens Studio,Lucena,45000.0,Basic,2022-01-15,4.3,13.93139,121.61722
1,Lights,BrightBeam Lights,Sariaya,38000.0,Standard,2022-02-20,4.5,13.96700,121.53300
2,Sounds,SoundWave Pro,Lucban,42000.0,Premium,2022-03-10,4.6,14.11333,121.55694
3,Decorator,Elegant Touch Decor,Pagbilao,50000.0,Deluxe,2022-04-22,4.7,13.97005,121.68700
4,Photographer,Memories Captured,Atimonan,47000.0,Standard,2022-05-18,4.4,13.99661,121.91805


In [5]:
venue.head()

,Venue Name,Municipality,Estimated Price,Package,Capacity,Date,Rating
0,Lucena Grand Pavilion,Lucena,152000.0,Large Ballroom,450.0,2022-03-12,4.7
1,Sariaya Heritage Hall,Sariaya,78500.0,Medium Hall,220.0,2022-07-08,4.5
2,Lucban Summit Center,Lucban,65000.0,Small Hall,120.0,2022-09-21,4.3
3,Gumaca Seaside Resort,Gumaca,98000.0,Outdoor Garden,300.0,2023-01-16,4.6
4,Pagbilao Bay Convention Center,Pagbilao,174500.0,Large Ballroom,500.0,2023-04-03,4.8


In [6]:
venue.head()

,Venue Name,Municipality,Estimated Price,Package,Capacity,Date,Rating
0,Lucena Grand Pavilion,Lucena,152000.0,Large Ballroom,450.0,2022-03-12,4.7
1,Sariaya Heritage Hall,Sariaya,78500.0,Medium Hall,220.0,2022-07-08,4.5
2,Lucban Summit Center,Lucban,65000.0,Small Hall,120.0,2022-09-21,4.3
3,Gumaca Seaside Resort,Gumaca,98000.0,Outdoor Garden,300.0,2023-01-16,4.6
4,Pagbilao Bay Convention Center,Pagbilao,174500.0,Large Ballroom,500.0,2023-04-03,4.8


In [7]:
cater.isnull().sum()

Company            31
Municipality       31
Estimated Price    31
Package            31
Dishes             31
Date               31
Rating             31
dtype: int64

In [8]:
vendors.isnull().sum()

Category           0
Vendor Name        0
Municipality       0
Estimated Price    0
Package            0
Date               0
Rating             0
Latitude           0
Longitude          0
dtype: int64

In [9]:
venue.isnull().sum()

Venue Name         3
Municipality       3
Estimated Price    3
Package            3
Capacity           3
Date               3
Rating             3
dtype: int64

In [10]:
cater.dropna(inplace=True)

In [11]:
vendors.dropna(inplace=True)

In [12]:
venue.dropna(inplace=True)

In [13]:
# the algorithmic backbone is a Content-Based Filtering model using Cosine Similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


features = ["Category", "Municipality", "Package", "Estimated Price"]
vendors_encoded = pd.get_dummies(vendors[features])

In [14]:
scaler = MinMaxScaler()
vendors_encoded[["Estimated Price"]] = scaler.fit_transform(vendors[["Estimated Price"]])

In [15]:
user_preferences = {
    "Category": "Photographer",
    "Municipality": "Tayabas",
    "Budget": 20000,
    "Package": "Standard"
}

In [16]:
user_vector = pd.DataFrame(0, index=[0], columns=vendors_encoded.columns)
if "Estimated Price" in user_vector.columns:
    user_vector["Estimated Price"] = scaler.transform([[user_preferences["Budget"]]])[0][0]

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [17]:
for col in [f"Category_{user_preferences['Category']}",
            f"Municipality_{user_preferences['Municipality']}",
            f"Package_{user_preferences['Package']}"]:
    if col in user_vector.columns:
        user_vector[col] = 1

# Compute similarity
similarities = cosine_similarity(user_vector, vendors_encoded)[0]
vendors["Similarity"] = similarities

In [18]:
recommendations = vendors.sort_values(["Similarity", "Rating"], ascending=[False, False]).head(5)
print(recommendations[["Vendor Name", "Category", "Municipality", "Package", "Estimated Price", "Rating", "Similarity"]])

             Vendor Name      Category Municipality   Package  \
111  PrismShine Lighting        Lights      Tayabas  Standard   
192    PrismLite Rentals        Lights      Tayabas  Standard   
172    LumaStar Lighting        Lights      Tayabas  Standard   
152      LumiWave Events        Lights      Tayabas  Standard   
24    LightCraft Studios  Photographer     Atimonan  Standard   

     Estimated Price  Rating  Similarity  
111          39000.0     4.4    0.615993  
192          39000.0     4.4    0.615993  
172          39500.0     4.4    0.612584  
152          41000.0     4.5    0.602031  
24           46000.0     4.4    0.563730  


In [19]:
import pandas as pd
import numpy as np

def recommend_vendors(df, user_prefs, top_n=5, weights=None, strict_service=True):
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371  # Earth radius in km
        phi1, phi2 = np.radians(lat1), np.radians(lat2)
        dphi = np.radians(lat2 - lat1)
        dlambda = np.radians(lon2 - lon1)
        a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
        return 2 * R * np.arcsin(np.sqrt(a))
    
    df = df.copy()
    # default weights
    if weights is None:
        weights = {'price':0.50, 'rating':0.20, 'package':0.15, 'municipality':0.15, 'category':0.10, 'distance':0.15}

    user_muni = user_prefs.get('Municipality')
    user_coords = coords[coords['Municipality'].str.lower() == user_muni.lower()]
    user_lat = user_coords['Latitude'].values[0]
    user_lon = user_coords['Longitude'].values[0]

    if 'Latitude' in df.columns and 'Longitude' in df.columns:
        df['Distance_km'] = df.apply(
            lambda row: haversine(user_lat, user_lon, row['Latitude'], row['Longitude']),
            axis=1
        )

    # 1) Optional hard-filter by Service (recommended)
    requested_service = user_prefs.get('Category', None)
    if requested_service and strict_service:
        mask = df['Category'].str.lower() == requested_service.lower()
        filtered = df[mask].copy()
        if filtered.empty:
            filtered = df.copy()
        df = filtered

    # If df became empty for other reasons, return empty
    if df.empty:
        return pd.DataFrame(columns=list(df.columns) + ['Score', 'Explanation'])

    # 2) Price similarity (normalized distance -> 0..1)
    budget = user_prefs.get('Budget', df['Estimated Price'].median())
    min_p, max_p = df['Estimated Price'].min(), df['Estimated Price'].max()
    denom = max_p - min_p if (max_p - min_p) > 0 else max_p + 1e-9
    price_sim = 1 - (np.abs(df['Estimated Price'] - budget) / denom)
    price_sim = price_sim.clip(0, 1)

    # 3) Rating normalized 0..1
    min_r, max_r = df['Rating'].min(), df['Rating'].max()
    rating_sim = (df['Rating'] - min_r) / (max_r - min_r + 1e-9)

    # 4) Package match (binary)
    pkg = user_prefs.get('Package', None)
    if pkg:
        pkg_match = (df['Package'].str.lower() == pkg.lower()).astype(float)
    else:
        pkg_match = np.ones(len(df))

    # Distance similarity (closer = higher score)
    if 'Distance_km' in df.columns:
        max_dist = df['Distance_km'].max()
        min_dist = df['Distance_km'].min()
        denom = max_dist - min_dist if (max_dist - min_dist) > 0 else max_dist + 1e-9
        dist_sim = 1 - ((df['Distance_km'] - min_dist) / denom)
    else:
        dist_sim = np.ones(len(df))

    # 5) Municipality match (binary)
    muni = user_prefs.get('Municipality', None)
    if muni:
        muni_match = (df['Municipality'].str.lower() == muni.lower()).astype(float)
    else:
        muni_match = np.ones(len(df))

    # 6) Service match (if not strict filtering)
    if not strict_service and requested_service:
        service_match = (df['Category'].str.lower() == requested_service.lower()).astype(float)
    else:
        service_match = np.ones(len(df))  # all 1 if strict_service applied

    # 7) Combine with weights
    score = (
        weights.get('price',0) * price_sim +
        weights.get('rating',0) * rating_sim +
        weights.get('package',0) * pkg_match +
        weights.get('municipality',0) * muni_match +
        weights.get('category',0) * service_match +
        weights.get('distance',0) * dist_sim
    )
    df['Score'] = score

    # 8) Build an explanation column
    explanations = []
    for idx, row in df.iterrows():
        parts = []
        parts.append(f"price_diff={int(abs(row['Estimated Price']-budget))}")
        parts.append(f"rating={row['Rating']:.1f}")
        if pkg:
            parts.append(f"package_match={'yes' if row['Package'].lower()==pkg.lower() else 'no'}")
        if muni:
            parts.append(f"municipality_match={'yes' if row['Municipality'].lower()==muni.lower() else 'no'}")
        if 'Distance_km' in row:
            parts.append(f"distance={row['Distance_km']:.1f}km")
        explanations.append('; '.join(parts))
    df['Explanation'] = explanations

    # 9) Return top-n sorted by Score, then Rating
    out = df.sort_values(['Score','Rating'], ascending=[False, False]).head(top_n)
    return out.reset_index(drop=True)

# Example usage:
vendors_df = pd.read_csv("vendors.csv", header=1)
prefs = {"Category":"Lights","Municipality":"Lucban","Budget":50000,"Package":"Standard"}
recs = recommend_vendors(vendors_df, prefs, top_n=5, weights=None, strict_service=True)
recs[['Vendor Name','Category','Municipality','Estimated Price','Rating','Score', 'Explanation']]

,Vendor Name,Category,Municipality,Estimated Price,Rating,Score,Explanation
0,BrightBeam Lights,Lights,Sariaya,38000.0,4.5,0.52,price_diff=12000; rating=4.5; package_match=ye...
1,Radiance Rentals,Lights,Candelaria,37000.0,4.5,0.52,price_diff=13000; rating=4.5; package_match=ye...
2,GlitterGlow Events,Lights,Sariaya,39500.0,4.5,0.52,price_diff=10500; rating=4.5; package_match=ye...
3,Luminary Events,Lights,Sariaya,40000.0,4.5,0.52,price_diff=10000; rating=4.5; package_match=ye...
4,RaySpark Rentals,Lights,Pagbilao,39500.0,4.5,0.52,price_diff=10500; rating=4.5; package_match=ye...


In [20]:
catering_df = pd.read_csv("catering_orders.csv")

catering_df["Municipality"].unique()

array(['Lucban', 'Lopez', 'Gumaca', 'Sariaya', 'Pagbilao', 'Real',
       'Tayabas', 'Lucena', nan, 'Unisan', 'Atimonan', 'Candelaria'],
      dtype=object)

In [21]:
venue_df = pd.read_csv("venue.csv", header=1)
venue_df["Municipality"].unique()

array(['Lucena', 'Sariaya', 'Lucban', 'Gumaca', 'Pagbilao', 'Atimonan',
       'Candelaria', 'Lopez', 'Tayabas', 'Unisan', nan], dtype=object)